# TP 0: Introducción al Web Scraping en Python

## 1. Realizar consultas HTTP en Python: _Requests_

[_Requests_](https://2.python-requests.org//es/latest/) es una librería Python que facilita el envio de consultas HTTP/1.1.

In [ ]:
import requests

In [ ]:
URL = "https://www.latercera.com/"

r = requests.get(URL)

In [ ]:
print(r.headers)

In [ ]:
print(r.headers['Content-Type'])
print(r.encoding)

In [ ]:
print(r.text)

Tutorial para uso más avanzado de _Requests_:
- _Requests_ quickstart [[1]](https://2.python-requests.org//es/latest/user/quickstart.html)
- uso avanzado de _Requests_ [[2]](https://2.python-requests.org//es/latest/user/advanced.html)

## 2. Extraer datos de documentos HTML en Python: _BeautifulSoup4_

[BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) es una librería Python que facilita la extracción de datos en archivos HTML y XML.

In [ ]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [ ]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(html_doc, 'html5lib')

In [ ]:
a_tags = bs.find_all('a')

for a_tag in a_tags:
    print(a_tag)

In [ ]:
for a_tag in a_tags:
    print(a_tag.text)

In [ ]:
for a_tag in a_tags:
    print(a_tag['href'])

In [ ]:
p_tags = bs.find_all('p',attrs = {'class':'title'})
for p_tag in p_tags:
    print(p_tag.text)

## 3. Extraer enlaces de noticias desde un medio de prensa: _Requests_ y _BeautifulSoup4_

**Nota Bene**: Multiplicar de forma automática las consultas HTTP hacia un sitio web es una mala práctica que no respeta las condiciones de uso del sitio. En ciertos paises, está prohibido por leyes.
Para evitar abusar las condiciones de uso de los sitios, cuidaremos no enviar más de 2 consultas por minuto hacia el mismo sitio.

Para evitar abusos, cada grupo de estudiantes utilizaran sitios de medios de prensa distintos:
- La Tercera: https://www.latercera.com/
- Cooperativa: https://www.cooperativa.cl/
- Bio Bio: https://www.biobiochile.cl/
- CNN Chile: https://www.cnnchile.com/
- 24 Horas TVN: https://www.24horas.cl/
- El País (España): https://elpais.com/elpais/portada_america.html
- El Mundo (españa): https://www.elmundo.es/
- 



In [ ]:
URL = "https://www.elmundo.es/ultimas-noticias.html"

r = requests.get(URL)

In [ ]:
print(r.text)

In [ ]:
bs = BeautifulSoup(r.text, 'html5lib')

a_tags = bs.find_all('a')

for a_tag in a_tags:
    print(a_tag['href'])

In [ ]:
h2_tags = bs.find_all('h2', attrs={"class":"mod-title"})

enlaces=[]

for h2_tag in h2_tags:
    specific_a_tags = h2_tag.find_all('a')
    for a_tag in specific_a_tags:
        print(a_tag['href'])
        enlaces.append(a_tag['href'])

## 4. Extraer contenidos de noticias

In [ ]:
import time

noticias=[]

for enlace in enlaces:
    
    time.sleep(2)
    print(enlace)
    r = requests.get(enlace)

    bs = BeautifulSoup(r.text, 'html5lib')

    div_noticia = bs.find('div', attrs={'class':'ue-l-article__body ue-c-article__body'})

    p_tags = div_noticia.find_all('p')

    texto_noticia=""
    try:
        for p_tag in p_tags:
            texto_noticia = texto_noticia+p_tag.text
            
    except:
        continue
    noticias.append((enlace,texto_noticia)) 

In [ ]:
noticias[2]

## 5. Aplicar procesos de Tratamiento Automático del Lenguaje

[SpaCy](https://spacy.io) es una librería Python que permite facilitar la realización de ciertos procesos de Tratamiento Automático del Lenguaje (TAL). Ejemplos de procesos de TAL son por ejemplos:
- la tokenización: segmentar el texto en palabras,
- la clasificación gramatical (pos): identificar si una palabra es un sustantivo, verbo, adjetivo, etc.
- la lematización: determinar la forma genérica de una palabra (por ej.: "superó" --> "superar", "ocasiones" --> "ocasión")

In [ ]:
import spacy
nlp = spacy.load('es_core_news_md')

In [ ]:
doc=nlp(noticias[1])

In [ ]:
for token in doc:
    print(token.text, token.pos_,token.lemma_)

In [ ]:
for token in doc:
    if (token.pos_ == "NOUN"):
        print(token.text, token.pos_,token.lemma_)

## 6. Almacenar los resultados en una base de datos: _MySQL_

In [ ]:
import mysql.connector

db_connection = mysql.connector.connect(user="root",host="localhost",password="root")
cursor = db_connection.cursor()

In [ ]:
##EJECUTAR 1 VEZ
#cursor.execute("CREATE DATABASE info229;")
cursor.execute("USE info229")
#cursor.execute("CREATE TABLE news (idNews INT PRIMARY KEY NOT NULL AUTO_INCREMENT, url VARCHAR(200), text TEXT);")


In [ ]:
for noticia in noticias:
    query="INSERT INTO news(url,text) VALUES (%s,%s)"
    cursor.execute(query,noticia)

cursor.execute("COMMIT");